In [ ]:
import math
import random
import time
from typing import Tuple, Optional
import sympy

In [ ]:
def generate_test_parameters(bit_length: int) -> Tuple[int, int, int]:
    """
    Generate test parameters for discrete logarithm computation.
    
    Args:
        bit_length (int): Desired bit length of the prime modulus
    
    Returns:
        Tuple[int, int, int]: (q, g, h) where:
            - q is a prime modulus
            - g is a generator of the multiplicative group Z*_q
            - h is a random element in the group
    """
    # Generate a random prime q of specified bit length
    q = sympy.randprime(2**(bit_length-1), 2**bit_length)
    
    # Find a generator g of the multiplicative group Z*_q
    def find_generator(q: int) -> int:
        if not sympy.isprime(q):
            raise ValueError("Modulus must be prime")
        
        # Find prime factors of q-1
        factors = sympy.factorint(q - 1)
        
        while True:
            g = random.randint(2, q-1)
            is_generator = True
            
            # Check if g is a generator
            for prime in factors:
                if pow(g, (q-1)//prime, q) == 1:
                    is_generator = False
                    break
            
            if is_generator:
                return g
    
    g = find_generator(q)
    
    # Generate random h = g^x mod q for some unknown x
    x = random.randint(1, q-2)
    h = pow(g, x, q)
    
    return q, g, h

def naive_algorithm(q: int, g: int, h: int) -> Optional[int]:
    """
    Implement the naive algorithm for discrete logarithm.
    
    Args:
        q (int): Prime modulus
        g (int): Generator of Z*_q
        h (int): Target element h = g^x mod q
    
    Returns:
        Optional[int]: x such that g^x ≡ h (mod q), or None if not found
    """
    for x in range(q-1):
        if pow(g, x, q) == h:
            return x
    return None

def baby_step_giant_step(q: int, g: int, h: int) -> Optional[int]:
    """
    Implement the Baby-Step Giant-Step algorithm.
    
    Args:
        q (int): Prime modulus
        g (int): Generator of Z*_q
        h (int): Target element h = g^x mod q
    
    Returns:
        Optional[int]: x such that g^x ≡ h (mod q), or None if not found
    """
    m = math.ceil(math.sqrt(q - 1))
    
    # Baby steps
    baby_steps = {pow(g, i, q): i for i in range(m)}
    
    # Precompute g^(-m)
    factor = pow(g, m * (q-2), q)
    
    # Giant steps
    for j in range(m):
        y = (h * pow(factor, j, q)) % q
        if y in baby_steps:
            return j * m + baby_steps[y]
    
    return None

def pollard_rho(q: int, g: int, h: int) -> Optional[int]:
    """
    Implement Pollard's Rho algorithm for discrete logarithm.
    
    Args:
        q (int): Prime modulus
        g (int): Generator of Z*_q
        h (int): Target element h = g^x mod q
    
    Returns:
        Optional[int]: x such that g^x ≡ h (mod q), or None if not found
    """
    def f(x_pair: Tuple[int, int, int]) -> Tuple[int, int, int]:
        x, a, b = x_pair
        subset = x % 3
        
        if subset == 0:
            x = (x * g) % q
            a = (a + 1) % (q - 1)
        elif subset == 1:
            x = (x * x) % q
            a = (2 * a) % (q - 1)
            b = (2 * b) % (q - 1)
        else:
            x = (x * h) % q
            b = (b + 1) % (q - 1)
            
        return (x, a, b)
    
    # Initialize starting points
    x_tortoise = (1, 0, 0)  # (x, a, b)
    x_hare = (1, 0, 0)      # (x, a, b)
    
    # Floyd's cycle-finding algorithm
    for _ in range(q):
        x_tortoise = f(x_tortoise)
        x_hare = f(f(x_hare))
        
        if x_tortoise[0] == x_hare[0]:
            r = (x_hare[1] - x_tortoise[1]) % (q - 1)
            s = (x_tortoise[2] - x_hare[2]) % (q - 1)
            
            if s == 0:
                return None
                
            # Solve r ≡ sx (mod q-1)
            x = (r * pow(s, -1, q-1)) % (q-1)
            
            if pow(g, x, q) == h:
                return x
                
            return None
            
    return None

def pohlig_hellman(q: int, g: int, h: int) -> Optional[int]:
    """
    Implement the Pohlig-Hellman algorithm.
    
    Args:
        q (int): Prime modulus
        g (int): Generator of Z*_q
        h (int): Target element h = g^x mod q
    
    Returns:
        Optional[int]: x such that g^x ≡ h (mod q), or None if not found
    """
    # Factor q-1 into prime powers
    factors = sympy.factorint(q - 1)
    
    # Chinese Remainder Theorem components
    x_values = []
    moduli = []
    
    for prime, exponent in factors.items():
        prime_power = prime ** exponent
        
        # Find x modulo prime_power
        g_p = pow(g, (q-1)//prime_power, q)
        h_p = pow(h, (q-1)//prime_power, q)
        
        # Use Pollard's Rho or Baby-Step Giant-Step for smaller subproblems
        if prime_power < 1000:
            x_p = baby_step_giant_step(q, g_p, h_p)
        else:
            x_p = pollard_rho(q, g_p, h_p)
            
        if x_p is None:
            return None
            
        x_values.append(x_p)
        moduli.append(prime_power)
    
    # Use Chinese Remainder Theorem to combine results
    def chinese_remainder_theorem(remainders, moduli):
        total = 0
        product = math.prod(moduli)
        
        for remainder, modulus in zip(remainders, moduli):
            p = product // modulus
            total += remainder * p * pow(p, -1, modulus)
            
        return total % product
    
    return chinese_remainder_theorem(x_values, moduli)

def number_field_sieve(q: int, g: int, h: int) -> Optional[int]:
    """
    Implement a simplified version of the Number Field Sieve algorithm.
    This is a basic implementation and doesn't include all optimizations.
    
    Args:
        q (int): Prime modulus
        g (int): Generator of Z*_q
        h (int): Target element h = g^x mod q
    
    Returns:
        Optional[int]: x such that g^x ≡ h (mod q), or None if not found
    """
    def find_polynomial_base(n: int, degree: int) -> Tuple[int, List[int]]:
        """Find a polynomial base m and coefficients for NFS."""
        m = int(pow(n, 1/(degree+1)))
        coeffs = []
        temp = n
        for _ in range(degree + 1):
            coeff = temp % m
            coeffs.append(coeff)
            temp //= m
        return m, coeffs

    def sieve_for_relations(bound: int, m: int, coeffs: List[int], q: int) -> List[Tuple[int, int]]:
        """Collect relations for the factor base."""
        relations = []
        for a in range(-bound, bound + 1):
            if a == 0:
                continue
            for b in range(1, bound + 1):
                if math.gcd(a, b) != 1:
                    continue
                    
                # Evaluate algebraic and rational numbers
                eval_rat = 0
                eval_alg = 0
                for i, coeff in enumerate(coeffs):
                    eval_rat = (eval_rat * m + coeff) % q
                    eval_alg = (eval_alg * a + b * coeff) % q
                
                if eval_rat != 0 and eval_alg != 0:
                    relations.append((eval_rat, eval_alg))
                
                if len(relations) >= bound:
                    return relations
        
        return relations

    # Only attempt NFS for larger numbers
    if q.bit_length() < 60:
        return None

    try:
        # Parameters for the simplified NFS
        degree = 4  # Polynomial degree
        bound = min(100, int(math.pow(q, 1/10)))  # Sieving bound
        
        # Find polynomial base
        m, coeffs = find_polynomial_base(q, degree)
        
        # Collect relations
        relations = sieve_for_relations(bound, m, coeffs, q)
        
        if not relations:
            return None
            
        # Linear algebra step (simplified)
        # In practice, this would involve solving a large sparse system
        # Here we just use the relations directly for a very basic approach
        for rel_g, rel_h in relations:
            # Try to reconstruct the discrete log
            if rel_g != 0 and rel_h != 0:
                possible_log = (rel_h * pow(rel_g, -1, q-1)) % (q-1)
                if pow(g, possible_log, q) == h:
                    return possible_log
        
        return None
        
    except Exception as e:
        print(f"NFS failed: {str(e)}")
        return None

import tracemalloc
import gc

def benchmark_algorithms(bit_lengths: list) -> None:
    """
    Benchmark the performance and memory usage of different discrete logarithm algorithms.
    
    Args:
        bit_lengths (list): List of bit lengths to test
    """
    algorithms = [
        ('Naive', naive_algorithm),
        ('Baby-Step Giant-Step', baby_step_giant_step),
        ('Pollard Rho', pollard_rho),
        ('Pohlig-Hellman', pohlig_hellman),
        ('Number Field Sieve', number_field_sieve)
    ]
    
    # Header for our output table
    print(f"{'Bit Length':<12} {'Algorithm':<20} {'Time (ms)':<12} {'Memory (KB)':<12} {'Success':<8}")
    print("-" * 64)
    
    for bits in bit_lengths:
        # Generate test parameters
        q, g, h = generate_test_parameters(bits)
        
        for name, algo in algorithms:
            # Skip algorithms for large bit lengths
            if (name == 'Naive' and bits > 24) or \
               (name == 'Baby-Step Giant-Step' and bits > 48):
                continue
            
            # Force garbage collection before each test
            gc.collect()
            
            # Start memory tracking
            tracemalloc.start()
            
            start_time = time.time()
            try:
                result = algo(q, g, h)
                end_time = time.time()
                
                # Get memory usage
                current, peak = tracemalloc.get_traced_memory()
                tracemalloc.stop()
                
                # Convert to milliseconds and kilobytes
                time_ms = (end_time - start_time) * 1000
                memory_kb = peak / 1024  # Convert bytes to KB
                
                success = result is not None and pow(g, result, q) == h
                print(f"{bits:<12} {name:<20} {time_ms:<12.2f} {memory_kb:<12.2f} {success}")
                
            except Exception as e:
                tracemalloc.stop()
                print(f"{bits:<12} {name:<20} Failed: {str(e)}")
            
            # Add theoretical complexity analysis
            theory_space = get_theoretical_space_complexity(name, q)
            if theory_space:
                print(f"{'':<12} {'(Theory)':<20} {'N/A':<12} {theory_space:<12}")
            
def get_theoretical_space_complexity(algo_name: str, q: int) -> str:
    """
    Get theoretical space complexity for each algorithm.
    
    Args:
        algo_name (str): Name of the algorithm
        q (int): Modulus value
    
    Returns:
        str: Description of theoretical space complexity
    """
    m = int(math.sqrt(q - 1))
    if algo_name == 'Naive':
        return "O(1)"
    elif algo_name == 'Baby-Step Giant-Step':
        return f"O(√n) ≈ {m} elements"
    elif algo_name == 'Pollard Rho':
        return "O(1)"
    elif algo_name == 'Pohlig-Hellman':
        # Space complexity depends on the largest prime factor
        factors = sympy.factorint(q - 1)
        largest_prime = max(factors.keys())
        return f"O(√p) ≈ {int(math.sqrt(largest_prime))} elements"
    elif algo_name == 'Number Field Sieve':
        # Simplified estimate for NFS space complexity
        return f"O(L[1/3]) ≈ {int(math.exp(math.pow(math.log(q), 1/3) * math.pow(math.log(math.log(q)), 2/3)))} elements"
    return ""

# Example usage
if __name__ == "__main__":
    # Test with different bit lengths
    bit_lengths = [4, 8, 16, 20, 24, 32, 40, 48, 56, 64]
    print("Starting benchmark test...")
    print("Note: Naive algorithm will be skipped for bit lengths > 24")
    print("      Baby-Step Giant-Step will be skipped for bit lengths > 48")
    print()
    benchmark_algorithms(bit_lengths)